# Date preprocessing

In [ ]:
!pip install -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [ ]:
from datasets import load_dataset

ds = load_dataset("pminervini/HaluEval", "qa")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.88k [00:00<?, ?B/s]

data-00000-of-00001.parquet:   0%|          | 0.00/3.75M [00:00<?, ?B/s]

Generating data split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    data: Dataset({
        features: ['knowledge', 'question', 'right_answer', 'hallucinated_answer'],
        num_rows: 10000
    })
})

In [ ]:
data = ds['data']

In [ ]:
sample = data.shuffle(seed=42).select(range(10))

In [ ]:
sample

Dataset({
    features: ['knowledge', 'question', 'right_answer', 'hallucinated_answer'],
    num_rows: 10
})

In [ ]:
# def build_cot_prompt(knowledge, question, answer):
#     return f"""
# You are an hallucination detector.

# I will give you 3 content which is Knowledge, question, answer, I want you to base on knowledge and justify whether the answer has halluciantion problem.

# Knowledge: {knowledge}

# Question: {question}

# Answer: {answer}

# Step-by-step reasoning:
# 1. Does the answer use any facts that are not in the knowledge?
# 2. Does the answer contradict any facts in the knowledge?
# 3. Is the answer factually aligned with the knowledge?

# I want you to answer only “1” if the answer contains hallucination, otherwise only “0”and tell me the reasons.



# """

# # only output “1” or “0”, donot contain other content.
# # and tell me the reasons

In [ ]:
def build_cot_prompt(knowledge, question, answer):
    return f"""
You are an expert hallucination detector. Analyze the given answer based on the provided knowledge and question.

Knowledge: {knowledge}

Question: {question}

Answer: {answer}

Please reason step-by-step:
1. Are there any facts in the answer that are not present in the knowledge?
2. Does the answer contradict the knowledge?
3. Is the answer factually aligned?

At the end, provide your final judgment "1" or "0" firstly:
Final Answer: "1" if the answer contains hallucination, otherwise "0".

If Final Answer is "0", explain briefly why the answer is factually correct.
"""


'\nYou are an expert hallucination detector. Analyze the given answer based on the provided knowledge and question.\n\nKnowledge: Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.First for Women is a woman\'s magazine published by Bauer Media Group in the USA.\n\nQuestion: Which magazine was started first Arthur\'s Magazine or First for Women?\n\nAnswer: First for Women was started first.\n\nPlease reason step-by-step:\n1. Are there any facts in the answer that are not present in the knowledge?\n2. Does the answer contradict the knowledge?\n3. Is the answer factually aligned?\n\nAt the end, provide your final judgment "1" or "0" firstly:\nFinal Answer: "1" if the answer contains hallucination, otherwise "0".\n\nIf Final Answer is "0", explain briefly why the answer is factually correct.\n'

# deepseek model_test

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import time

# model_name = "Qwen/Qwen3-14B"
model_name = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id



Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "user", "content": prompt}

]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
# tokenizer.pad_token_id = tokenizer.eos_token_id  # ✅ 添加这行
outputs = model.generate(input_tensor.to(model.device), max_new_tokens=500)
result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
print("-"*40)
print(result)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


----------------------------------------
1


In [ ]:
result_storage = []
result_storage.append(result)

In [ ]:
result_storage

['1']

# Qwen mdoel_test

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen3-14B"

In [ ]:
# ✅ 加载模型和 tokenizer（Qwen 必须加 trust_remote_code=True）
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True)



Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# ✅ 用消息格式构造 prompt
messages = [
    {"role": "system", "content": prompt}
]

# ✅ 使用 Qwen 的 chat_template
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

# ✅ 生成模型输出
outputs = model.generate(inputs, max_new_tokens=200)
response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

print(response)


</think>

1. **Are there any facts in the answer that are not present in the knowledge?**  
   The answer claims that *First for Women* was started first. However, the knowledge states that *Arthur's Magazine* was published from 1844–1846, while *First for Women* is a modern magazine (no specific start date is given in the knowledge, but it is implied to be much later, as it is a 21st-century publication). The answer introduces a claim (*First for Women was started first*) that is not explicitly stated in the knowledge but can be inferred from the timeline.

2. **Does the answer contradict the knowledge?**  
   The knowledge does not explicitly state when *First for Women* was started, but it does provide a clear timeframe for *Arthur's Magazine* (1844–1846). Given that *First for Women* is described as a modern magazine and not tied to
